# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output/city_weather.csv", index_col=0)
cities_df

,city,country,lat,lon,temp,temp_max,temp_min,humidity,cloudness,wind
0,Saint Helena,SH,-15.9333,-5.7167,67.86,67.86,67.86,84,66,16.17
1,Tôlanaro,MG,-25.0333,47.0000,78.80,78.80,78.80,89,40,6.91
2,Ushuaia,AR,-54.8000,-68.3000,51.80,51.80,51.80,76,40,4.61
3,Cabedelo,BR,-6.9667,-34.8333,77.00,77.00,77.00,88,20,3.44
4,Volovo,RU,52.0266,37.8854,30.15,30.15,30.15,98,100,17.85
...,...,...,...,...,...,...,...,...,...,...
589,Belmonte,BR,-15.8500,-38.9000,77.00,77.00,77.00,83,75,10.36
590,São Francisco,BR,-15.9500,-44.8667,70.54,70.54,70.54,91,98,3.31
591,Ketchikan,US,55.3422,-131.6461,39.20,39.20,39.20,87,20,8.05
592,Hukuntsi,BW,-24.0000,21.7500,75.16,75.16,75.16,59,21,10.67


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# use for test api
#cities_df = cities.head(30)
#cities_df

In [4]:
# map_type = {"HYBRID","ROADMAP","SATELLITE","HYBRID","TERRAIN"}
gmaps.configure(api_key=g_key)
fig = gmaps.figure(layout={'width': '960px', 'height': '800px'}, map_type="SATELLITE")
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['lat', 'lon']], weights=cities_df['humidity'], max_intensity=cities_df['humidity'].max().astype(float), point_radius=3.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='800px', width='960px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# conditions:
# 1. A max temperature lower than 82 degrees but higher than 72. (22°C - 28°C)
# 2. Wind speed less than 10 mph.
# 3. less than 50% cloudiness.
suitable_cities = cities_df.loc[(cities_df["temp_max"] <= 82) & (cities_df["temp_max"] >= 72) & (cities_df["cloudness"] < 50) & (cities_df["wind"] <10)]
suitable_cities

,city,country,lat,lon,temp,temp_max,temp_min,humidity,cloudness,wind
1,Tôlanaro,MG,-25.0333,47.0000,78.80,78.80,78.80,89,40,6.91
3,Cabedelo,BR,-6.9667,-34.8333,77.00,77.00,77.00,88,20,3.44
6,Victoria,SC,-4.6167,55.4500,77.00,77.00,77.00,94,20,2.30
19,Bonthe,SL,7.5264,-12.5050,77.54,77.54,77.54,87,0,3.76
37,Mersing,MY,2.4312,103.8405,78.80,78.80,78.80,83,40,6.91
40,Libreville,GA,0.3833,9.4500,73.40,73.40,73.40,100,20,2.30
64,Cap Malheureux,MU,-19.9842,57.6142,79.00,79.00,79.00,78,6,4.00
81,Tsiombe,MG,-25.3000,45.4833,75.16,75.16,75.16,94,20,5.37
96,Inhambane,MZ,-23.8650,35.3833,77.18,77.18,77.18,88,0,5.32
107,Neuquén,AR,-38.9516,-68.0591,75.20,75.20,75.20,60,40,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Nearby Search url generating
# location=lat,lon
# radius={meter}
# type=lodging
def get_url(parameters):
    param = ""
    for key in parameters.keys():
        param = param + f"{key}={parameters[key]}&"
    return f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?{param}key={g_key}"

In [7]:
hotels = []
for index, row in suitable_cities.iterrows():
    url = get_url({
        "location" : f"{row['lat']},{row['lon']}",
        "type" : "lodging",
        "radius" : 5000
        })
#    print(url)
    response = requests.get(url)
    data = response.json()
    try:
        hotels.append({
            "index" : index,
            "Hotel Name" : data["results"][0]["name"]
        })
    except:
        hotels.append({
            "index" : index,
            "Hotel Name" : "No hotel found."
        })

In [8]:
hotels
hotel_df = pd.DataFrame(hotels).set_index("index", drop=True)
hotel_df.index.name = None
hotel_df = hotel_df.merge(suitable_cities, left_index=True, right_index=True)
hotel_df

,Hotel Name,city,country,lat,lon,temp,temp_max,temp_min,humidity,cloudness,wind
1,Lavasoa,Tôlanaro,MG,-25.0333,47.0000,78.80,78.80,78.80,89,40,6.91
3,Lua Mansa Pousada,Cabedelo,BR,-6.9667,-34.8333,77.00,77.00,77.00,88,20,3.44
6,Sunset Beach Hotel,Victoria,SC,-4.6167,55.4500,77.00,77.00,77.00,94,20,2.30
19,Bonthe Holiday Village,Bonthe,SL,7.5264,-12.5050,77.54,77.54,77.54,87,0,3.76
37,Timotel Hotel,Mersing,MY,2.4312,103.8405,78.80,78.80,78.80,83,40,6.91
40,Hotel Re-Ndama,Libreville,GA,0.3833,9.4500,73.40,73.40,73.40,100,20,2.30
64,Pereybere Hotel & Spa,Cap Malheureux,MU,-19.9842,57.6142,79.00,79.00,79.00,78,6,4.00
81,No hotel found.,Tsiombe,MG,-25.3000,45.4833,75.16,75.16,75.16,94,20,5.37
96,Oceano,Inhambane,MZ,-23.8650,35.3833,77.18,77.18,77.18,88,0,5.32
107,Hotel del Comahue,Neuquén,AR,-38.9516,-68.0591,75.20,75.20,75.20,60,40,6.91


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
symbols_layer = gmaps.symbol_layer(hotel_df[["lat","lon"]], fill_color='white', stroke_color='red', info_box_content=hotel_info)
fig.add_layer(symbols_layer)
fig

Figure(layout=FigureLayout(height='800px', width='960px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
